In [ ]:
# STEP 1: Monta Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# STEP 2: Installa dcm2niix
!sudo apt-get install -y dcm2niix


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libyaml-cpp0.7
The following NEW packages will be installed:
  dcm2niix libyaml-cpp0.7
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 354 kB of archives.
After this operation, 1,231 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libyaml-cpp0.7 amd64 0.7.0+dfsg-8build1 [97.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 dcm2niix amd64 1.0.20211006-1build1 [256 kB]
Fetched 354 kB in 1s (318 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controllin

In [ ]:
# STEP 3: Imposta le cartelle
import os
import glob

# Directory principale con le cartelle ADNI
base_dir = "/content/drive/MyDrive/app_project/mri_ultimostr"

# Directory unica di output
output_dir = "/content/drive/MyDrive/NIFTI_OUTPUT_str"
os.makedirs(output_dir, exist_ok=True)


In [ ]:
# STEP 4: Scansione e conversione
import subprocess
import shutil  # Import shutil


# Cicla tutte le cartelle ADNI
for adni_folder in sorted(os.listdir(base_dir)):
    adni_path = os.path.join(base_dir, adni_folder)
    if not os.path.isdir(adni_path): continue

    # Dentro ogni cartella ADNI c'è una sola cartella: l'ID del paziente
    patient_folders = os.listdir(adni_path)
    for patient_id in patient_folders:
        mp_rage_path = os.path.join(adni_path, patient_id, "MP-RAGE")
        if not os.path.isdir(mp_rage_path): continue

        # Ogni sotto-cartella è una data
        for date_folder in os.listdir(mp_rage_path):
            dicom_path = os.path.join(mp_rage_path, date_folder)
            if not os.path.isdir(dicom_path): continue

            # Output file name
            output_name = f"{patient_id}_{date_folder.replace('/', '-')}"  # Sicurezza nome file
            temp_output_dir = "/content/temp_dcm2niix"
            os.makedirs(temp_output_dir, exist_ok=True)

            # Converti con dcm2niix
            subprocess.run([
                "dcm2niix",
                "-z", "y",                      # Comprime in .nii.gz
                "-f", output_name,             # Nome file finale
                "-o", temp_output_dir,         # Output temporaneo
                dicom_path
            ])

            # Sposta il risultato nella cartella finale
            for file in os.listdir(temp_output_dir):
                if file.endswith(".nii.gz"):
                    shutil.move(  # Use shutil.move instead of os.rename
                        os.path.join(temp_output_dir, file),
                        os.path.join(output_dir, file)
                    )
            # Pulisci la cartella temporanea
            for f in os.listdir(temp_output_dir):
                os.remove(os.path.join(temp_output_dir, f))


TRASFORMARE UN FILE STAT ESTRANDO LA COLONNA DI VOLUME_MM3

In [ ]:
import os
import pandas as pd
from collections import defaultdict
from datetime import datetime

# Percorso principale
base_path = "/content/drive/MyDrive/app_project/fastsurfer_seg"

# Dizionario: paziente -> lista di (data, volume_dict)
pazienti = defaultdict(list)

# Per verificare strutture uniche per file
strutture_per_file = {
    "aseg+DKT.stats": set(),
    "cerebellum.CerebNet.stats": set(),
    "hypothalamus.HypVINN.stats": set()
}

# Funzione per leggere file .stats
def leggi_stats(file_path, file_label):
    volume_dict = {}
    if not os.path.exists(file_path):
        return volume_dict
    with open(file_path, "r") as f:
        for line in f:
            if line.startswith("#") or line.strip() == "":
                continue
            fields = line.split()
            if len(fields) >= 5:
                struct = fields[4]
                try:
                    volume = float(fields[3])
                    volume_dict[struct] = volume  # <-- senza prefisso nel nome colonna
                    strutture_per_file[file_label + ".stats"].add(struct)  # <-- mantieni il prefisso solo per controllo
                except:
                    continue
    return volume_dict

for folder_name in os.listdir(base_path):
    folder_path = os.path.join(base_path, folder_name)
    stats_dir = os.path.join(folder_path, "stats")

    if not os.path.exists(stats_dir):
        continue

    try:
        parts = folder_name.split("_")
        patient_id = "_".join(parts[:3])  # es: 003_S_1122
        date_str = parts[3]
        date_obj = datetime.strptime(date_str, "%Y-%m-%d")

        # Unisci tutti e tre i dizionari
        volume_dict = {}
        volume_dict.update(leggi_stats(os.path.join(stats_dir, "aseg+DKT.stats"), "aseg+DKT"))
        volume_dict.update(leggi_stats(os.path.join(stats_dir, "cerebellum.CerebNet.stats"), "cerebellum.CerebNet"))
        volume_dict.update(leggi_stats(os.path.join(stats_dir, "hypothalamus.HypVINN.stats"), "hypothalamus.HypVINN"))

        if volume_dict:  # Solo se almeno uno dei tre è stato letto con successo
            pazienti[patient_id].append((date_obj, volume_dict))

    except Exception as e:
        print(f"Errore in {folder_name}: {e}")
        continue

# Trova il massimo numero di visite
max_visite = max(len(v) for v in pazienti.values())

# Per ogni posizione temporale (prima visita, seconda, ecc.)
visite_posizionali = defaultdict(list)

for i in range(max_visite):
    dati_per_visita = {}

    for pid, visite in pazienti.items():
        visite_sorted = sorted(visite, key=lambda x: x[0])
        if i < len(visite_sorted):
            data, vol_dict = visite_sorted[i]
            vol_dict["Data"] = data.strftime("%Y-%m-%d")
            dati_per_visita[pid] = vol_dict

    df = pd.DataFrame.from_dict(dati_per_visita, orient="index")
    df.index.name = "Patient_ID"
    colonne = ['Data'] + [c for c in df.columns if c != 'Data']
    df = df[colonne]
    visite_posizionali[f"Visita_{i+1}"] = df

# Salva in Excel
excel_path = "/content/volumi_temporali_final.xlsx"
with pd.ExcelWriter(excel_path) as writer:
    for sheet_name, df in visite_posizionali.items():
        df.to_excel(writer, sheet_name=sheet_name)

# Controllo di sovrapposizione delle strutture tra file
all_duplicates = False
labels = list(strutture_per_file.keys())
for i in range(len(labels)):
    for j in range(i+1, len(labels)):
        common = strutture_per_file[labels[i]].intersection(strutture_per_file[labels[j]])
        if common:
            print(f"\n⚠️ Colonne comuni tra {labels[i]} e {labels[j]}:")
            for c in common:
                print(f" - {c}")
            all_duplicates = True

if not all_duplicates:
    print("\n✅ Nessuna colonna in comune tra i tre file stats.")

# Scarica
from google.colab import files
files.download(excel_path)



⚠️ Colonne comuni tra aseg+DKT.stats e cerebellum.CerebNet.stats:
 - Right-Cerebellum-Cortex
 - Left-Cerebellum-Cortex
 - Right-Cerebellum-White-Matter
 - Left-Cerebellum-White-Matter


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

SELEZIONIAMO LE 70 FEATURE PIU RILEVANTI SECONDO LA LETTERATURA SCIENTIFICA


In [ ]:
# Definisci le strutture da estrarre
strutture_interesse = [
    'Left-Hippocampus', 'Right-Hippocampus', 'ctx-lh-entorhinal', 'ctx-rh-entorhinal', 'L-Fornix',
    'ctx-rh-posteriorcingulate', 'ctx-lh-posteriorcingulate', 'ctx-lh-parahippocampal', 'ctx-rh-parahippocampal',
    'R-Fornix', 'ctx-rh-superiortemporal', 'Left-Amygdala', 'Right-Amygdala', 'ctx-rh-middletemporal',
    'ctx-rh-fusiform', 'Left-Thalamus', 'Right-Thalamus', 'ctx-rh-insula', 'ctx-lh-insula',
    'ctx-lh-rostralanteriorcingulate', 'ctx-lh-fusiform', 'ctx-rh-inferiortemporal', 'ctx-lh-middletemporal',
    'ctx-lh-superiortemporal', 'ctx-lh-inferiortemporal', 'ctx-rh-rostralanteriorcingulate',
    'Left-Cerebral-White-Matter', 'Right-Cerebral-White-Matter', 'ctx-rh-precuneus', 'ctx-rh-supramarginal',
    'ctx-lh-precuneus', 'ctx-lh-parstriangularis', 'ctx-rh-isthmuscingulate', 'ctx-lh-caudalanteriorcingulate',
    'ctx-lh-isthmuscingulate', 'ctx-rh-parstriangularis', 'ctx-lh-supramarginal', 'ctx-rh-medialorbitofrontal',
    'ctx-lh-medialorbitofrontal', 'ctx-lh-inferiorparietal', 'ctx-rh-caudalanteriorcingulate',
    'ctx-lh-superiorfrontal', 'ctx-rh-superiorfrontal', 'ctx-rh-superiorparietal', 'ctx-rh-parsopercularis',
    'R-C.mammilare', 'L-C.mammilare', 'WM-Hypointensities', 'ctx-rh-inferiorparietal', 'ctx-lh-superiorparietal',
    'ctx-lh-parsopercularis', 'ctx-rh-caudalmiddlefrontal', 'ctx-rh-parsorbitalis', 'ctx-lh-transversetemporal',
    'ctx-rh-rostralmiddlefrontal', 'ctx-lh-rostralmiddlefrontal', 'CC_Anterior', 'CSF', 'ctx-lh-caudalmiddlefrontal',
    'ctx-rh-lateralorbitofrontal', 'ctx-lh-lateralorbitofrontal', 'ctx-lh-parsorbitalis', 'ctx-rh-transversetemporal',
    'Right-Inf-Lat-Vent', 'Third-Ventricle', '3rd-Ventricle', 'Right-Lateral-Ventricle',
    'Left-Lateral-Ventricle', 'Left-Inf-Lat-Vent', 'CC_Mid_Anterior', 'CC_Central'
]

# Nuovo dizionario per i dati filtrati
visite_filtrate = {}

for nome_visita, df in visite_posizionali.items():
    colonne_da_tenere = ['Data'] + [c for c in df.columns if c in strutture_interesse]
    df_filtrato = df[colonne_da_tenere]
    visite_filtrate[nome_visita] = df_filtrato

# Salva in Excel
output_path = "/content/strutture_selezionate_solo.xlsx"
with pd.ExcelWriter(output_path) as writer:
    for nome_visita, df in visite_filtrate.items():
        df.to_excel(writer, sheet_name=nome_visita)

# Download (opzionale in Colab)
from google.colab import files
files.download(output_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

UNIAMO I DATASET PROVENTI DAI VALORI TABULARI E IL DATASET GNENOMICO


In [ ]:
import pandas as pd

# Percorsi dei file
genomic_file = "/content/drive/MyDrive/app_project/genomic_dataset.xlsx"
mri_file = "/content/drive/MyDrive/app_project/Dataset_tabs.xlsx"
output_file = "/content/drive/MyDrive/app_project/Dataset_tabs+genomica.xlsx"

# Carica i dati genomici
df_genomic = pd.read_excel(genomic_file)
df_genomic.columns = df_genomic.columns.str.strip()  # Pulisce spazi bianchi se presenti

# Carica tutti i fogli MRI
mri_data = pd.read_excel(mri_file, sheet_name=None)

# Dizionario per i nuovi fogli combinati
merged_data = {}

# Unisci i dati genomici a ciascun foglio MRI
for sheet_name, df_visita in mri_data.items():
    df_visita.columns = df_visita.columns.str.strip()  # Pulisce nomi colonne anche qui
    merged_df = pd.merge(df_visita, df_genomic, on="Subject_ID", how="left")
    merged_data[sheet_name] = merged_df

# Scrivi il nuovo file Excel combinato
with pd.ExcelWriter(output_file) as writer:
    for sheet_name, df in merged_data.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("✅ File combinato creato con successo:", output_file)

# Scarica da Colab
from google.colab import files
files.download(output_file)


✅ File combinato creato con successo: /content/drive/MyDrive/app_project/Dataset_tabs+genomica.xlsx


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Lista per raccogliere i pazienti mancanti
pazienti_mancanti = {}

for sheet_name, df in merged_data.items():
    # Seleziona pazienti con tutti valori genomici nulli
    # Esclude la colonna 'Subject_ID' e controlla solo le colonne genomiche
    genomics_columns = df.columns.difference(df.columns[:df.columns.get_loc('Subject_ID') + 1])
    missing = df[df[genomics_columns].isnull().all(axis=1)]["Subject_ID"].tolist()

    if missing:
        pazienti_mancanti[sheet_name] = missing

# Stampa risultati
if pazienti_mancanti:
    print("⚠️ I seguenti pazienti non hanno dati genomici associati:")
    for sheet, patients in pazienti_mancanti.items():
        print(f"- {sheet}: {len(patients)} pazienti mancanti")
        print("  ", patients)
else:
    print("✅ Tutti i pazienti hanno dati genomici associati.")


✅ Tutti i pazienti hanno dati genomici associati.


CONCATENO I DUE CAZZI DI FILE DI GENOMIC IN UNICO FILE

In [ ]:
# Step 1: Carica le librerie
import pandas as pd

df1 =  pd.read_excel("/content/drive/MyDrive/app_project/genomic_dataset.xlsx")
df2 =  pd.read_excel("/content/drive/MyDrive/app_project/genomic_dataset_2.xlsx")

# Estrai i nomi delle colonne
cols1 = set(df1.columns)
cols2 = set(df2.columns)

# Trova colonne comuni e differenze
common_cols = cols1 & cols2
only_in_df1 = cols1 - cols2
only_in_df2 = cols2 - cols1

# Stampa i risultati
print("✅ Colonne in comune:")
print(sorted(common_cols))

print("\n❌ Colonne presenti solo in genomic_dataset.xlsx:")
print(sorted(only_in_df1))

print("\n❌ Colonne presenti solo in genomic_dataset_2.xlsx:")
print(sorted(only_in_df2))


✅ Colonne in comune:
['Subject_ID', 'rs11136000_T', 'rs11767557_C', 'rs11771145_A', 'rs17125944_C', 'rs3764650_G', 'rs3851179_A', 'rs3865444_T', 'rs610932_A', 'rs744373_C']

❌ Colonne presenti solo in genomic_dataset.xlsx:
['rs190982_G', 'rs7412_T']

❌ Colonne presenti solo in genomic_dataset_2.xlsx:
[]


In [ ]:

# Trova le colonne comuni
common_cols = list(set(df1.columns) & set(df2.columns))

# Ordina le colonne nello stesso ordine di df1
common_cols_sorted = [col for col in df1.columns if col in common_cols]

# Seleziona solo le colonne comuni con ordine coerente
df1_common = df1[common_cols_sorted]
df2_common = df2[common_cols_sorted]

# Concatena
df_combined = pd.concat([df1_common, df2_common], ignore_index=True)

# Salva il risultato
output_file = 'genomic_dataset_combined_common_columns.xlsx'
df_combined.to_excel(output_file, index=False)

print(f"✅ File combinato salvato come '{output_file}' con {len(df_combined)} righe e {len(common_cols_sorted)} colonne comuni.")


✅ File combinato salvato come 'genomic_dataset_combined_common_columns.xlsx' con 3017 righe e 10 colonne comuni.


PASSO FINALE PER AVERE IL DATASET COMPLETO DI RISONANZE MAGNETICHE E DATI GENETICI DEL PAZIENTE


In [ ]:
import pandas as pd

# === Step 1: Carica i file ===

alz_dict =  pd.read_excel("/content/drive/MyDrive/app_project/Alzhameir_dataset.xlsx", sheet_name=None)
gene_df =  pd.read_excel("/content/drive/MyDrive/app_project/Gene_dataset.xlsx")


# === Step 2: Rimuovi duplicati nel Gene_dataset ===
gene_df = gene_df.drop_duplicates(subset='Subject_ID')

# === Step 3: Verifica che tutti gli Subject_ID siano presenti anche nel Gene_dataset ===
gene_subject_ids = set(gene_df['Subject_ID'])

missing_ids_total = {}

for sheet_name, df in alz_dict.items():
    alz_subject_ids = set(df['Subject_ID'])
    missing_ids = alz_subject_ids - gene_subject_ids
    if missing_ids:
        missing_ids_total[sheet_name] = missing_ids

if missing_ids_total:
    print("❌ Alcuni Subject_ID nei fogli Alzheimer NON sono presenti nel Gene_dataset:")
    for sheet, ids in missing_ids_total.items():
        print(f" - Foglio '{sheet}': {len(ids)} mancanti")
    # Se vuoi forzare il merge anche con soggetti mancanti, puoi farlo aggiungendo `how='left'` più avanti
else:
    print("✅ Tutti i Subject_ID nei fogli Alzheimer sono presenti nel Gene_dataset.")

    # === Step 4: Esegui il merge per ogni foglio ===
    updated_sheets = {}

    for sheet_name, df in alz_dict.items():
        merged_df = df.merge(gene_df, on='Subject_ID', how='left')
        updated_sheets[sheet_name] = merged_df

    # === Step 5: Salva in un nuovo file Excel con tutti i fogli aggiornati ===
    output_file = 'Alzhameir_dataset_with_genes.xlsx'
    with pd.ExcelWriter(output_file) as writer:
        for sheet_name, df in updated_sheets.items():
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"\n✅ File salvato come '{output_file}' con le feature genetiche aggiunte.")


✅ Tutti i Subject_ID nei fogli Alzheimer sono presenti nel Gene_dataset.

✅ File salvato come 'Alzhameir_dataset_with_genes.xlsx' con le feature genetiche aggiunte.


In [ ]:
import pandas as pd

# === Step 1: Carica il file già unito (con dati genetici) ===
alz_dict =  pd.read_excel("/content/drive/MyDrive/app_project/Alzhemeir_Dataset.xlsx", sheet_name=None)

# === Step 2: Aggiungi colonna per ordinare le visite ===
visit_order = list(alz_dict.keys())
for i, sheet in enumerate(visit_order):
    alz_dict[sheet]['__Visit_Order__'] = i  # serve per ordinare temporalmente

# === Step 3: Unisci i fogli in un unico DataFrame ===
df_all = pd.concat(alz_dict.values(), ignore_index=True)

# === Step 4: Ordina per paziente e visita
df_all_sorted = df_all.sort_values(by=['Subject_ID', '__Visit_Order__']).copy()

# === Step 5: Individua solo le colonne numeriche con NaN
num_cols_with_nan = [col for col in df_all_sorted.select_dtypes(include='number').columns
                     if df_all_sorted[col].isna().any() and col != '__Visit_Order__']

print(f"🔍 Colonne numeriche con valori mancanti da interpolare: {num_cols_with_nan}")

# === Step 6: Interpolazione SOLO dove serve, SOLO per quei pazienti e colonne ===
for col in num_cols_with_nan:
    df_all_sorted[col] = df_all_sorted.groupby('Subject_ID')[col].transform(
        lambda group: group.interpolate(method='linear', limit_direction='both')
    )

# === Step 7: Ricostruisci i fogli originali ===
final_sheets = {}
for i, sheet_name in enumerate(visit_order):
    df_sheet = df_all_sorted[df_all_sorted['__Visit_Order__'] == i].drop(columns='__Visit_Order__')
    final_sheets[sheet_name] = df_sheet.reset_index(drop=True)

# === Step 8: Salva il file Excel finale ===
output_file = 'Alzhameir_dataset_with_genes_interpolated.xlsx'
with pd.ExcelWriter(output_file) as writer:
    for sheet_name, df in final_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"\n✅ Interpolazione completata SOLO sui valori nulli. File salvato come '{output_file}'.")


🔍 Colonne numeriche con valori mancanti da interpolare: ['L-Fornix', 'R-Fornix', 'R-C.mammilare', 'L-C.mammilare', 'Third-Ventricle']

✅ Interpolazione completata SOLO sui valori nulli. File salvato come 'Alzhameir_dataset_with_genes_interpolated.xlsx'.
